In [0]:
from theano import sandbox

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

In [0]:
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l1, l2
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam

In [17]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [18]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [19]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('Total Chracters:', vocab_size)

Total Chracters: 58


In [0]:
chars.insert(0, '\0')

In [22]:
''.join(chars)

'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
idx = [char_indices[c] for c in text]

In [25]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [26]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

### Preprocess and create model

In [27]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen + 1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb_sequences:', len(sentences))

nb_sequences: 600854


In [0]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [29]:
sentences.shape, next_chars.shape

((600852, 40), (600852, 40))

In [31]:
sentences[0]

array([43, 45, 32, 33, 28, 30, 32,  1,  1,  1, 46, 48, 43, 43, 42, 46, 36,
       41, 34,  2, 47, 35, 28, 47,  2, 47, 45, 48, 47, 35,  2, 36, 46,  2,
       28,  2, 50, 42, 40, 28])

In [34]:
next_chars[0]

array([45, 32, 33, 28, 30, 32,  1,  1,  1, 46, 48, 43, 43, 42, 46, 36, 41,
       34,  2, 47, 35, 28, 47,  2, 47, 45, 48, 47, 35,  2, 36, 46,  2, 28,
        2, 50, 42, 40, 28, 41])

In [32]:
''.join(indices_char[i] for i in sentences[0])

'preface\n\n\nsupposing that truth is a woma'

In [33]:
''.join(indices_char[i] for i in next_chars[0])

'reface\n\n\nsupposing that truth is a woman'

In [0]:
n_fac = 42

In [36]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=maxlen),
    LSTM(512, input_dim=n_fac, return_sequences=True, dropout_U=0.2, dropout_W=0.2, consume_less='gpu'),
    Dropout(0.2),
    LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2, consume_less='gpu'),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')
])

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, input_shape=(None, 42), dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  """


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [0]:
def print_example():
    seed_string = 'ethics is a basic foundation of all that'
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [39]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/1
600852/600852 [==============================] - 1025s 2ms/step - loss: 1.4815


In [40]:
print_example()

ethics is a basic foundation of all that will which considered there is
not mistaken. it is no way of "-natural"--guld fting that is dangerous and changed that we are
contradictively finally speak of women, who,
has not hitherto no existence and politics, in control of the new, no
organic and more than this other philosophy, but not--what does plainly be
gui


In [0]:
model.optimizer.lr=0.0001

In [42]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
    64/600852 [..............................] - ETA: 23:42 - loss: 1.2611

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


600852/600852 [==============================] - 1028s 2ms/step - loss: 1.2791


In [43]:
print_example()

ethics is a basic foundation of all that
is "each other." the
ascetic soul was proceeding
to it: from
the kind of
invention,
     which books upon further, evilscent rank, just the mother and consequences of
good and way through the seems to
understand himself--or -lett, indeed, industry, the old philosopher with other spirits; they _appear i movement that f


In [0]:

model.optimizer.lr=1e-2

In [46]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
    64/600852 [..............................] - ETA: 24:31 - loss: 1.2496

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


600852/600852 [==============================] - 1016s 2ms/step - loss: 1.2467


In [47]:
print_example()

ethics is a basic foundation of all that the realm of that
things from a bell, to assust
to him.

231. when people again our
problem of rank; for the classes of their very requirements than any
kind of his own
teacher, "matter" about
it by the science
indeed all the meanwhile retrouded the
soul, the riddle of the "soul," had to give
ourselves, in order to do
